In [0]:
dbutils.fs.rm("dbfs:/silver/weather/", recurse=True)

In [0]:
from pyspark.sql import functions as F
import datetime as dt

# Dynamic date
INGESTION_DATE = dt.date.today().isoformat()

# Input from partitioned Bronze
BRONZE_PATH = "dbfs:/bronze/weather/"

# Output path (shared, partitioned Silver)
SILVER_PATH = "dbfs:/silver/weather/"


In [0]:
# Read today's weather only
df_bronze = (
    spark.read.format("delta")
    .load(BRONZE_PATH)
    .filter(F.col("ingestion_date") == INGESTION_DATE)
)

df_bronze.select("forecast_time", "temperature", "shortForecast", "windSpeed").show(5, truncate=False)


In [0]:
df_silver = (
    df_bronze.select(
        F.col("forecast_time"),
        F.col("temperature").cast("int"),
        F.col("shortForecast").alias("condition"),
        F.col("windSpeed"),
        F.col("forecast_retrieved_at")
    )
    .withColumn("processed_at", F.current_timestamp())
    .withColumn("ingestion_date", F.lit(INGESTION_DATE))
    .dropDuplicates(["forecast_time"])
)


In [0]:
df_silver.write \
    .format("delta") \
    .mode("append") \
    .partitionBy("ingestion_date") \
    .save(SILVER_PATH)

print("✓ Weather Silver table saved (partitioned append)")
